# 02. Data Cleaning & IntegrationObjective: Clean text fields, parse dates, handle missing values, and merge datasets.

In [ ]:
import pandas as pdimport numpy as npimport os

## 1. Load Raw Data

In [ ]:
raw_enc = pd.read_csv('data/enrolment_sample.csv')raw_bio = pd.read_csv('data/biometric_sample.csv')raw_demo = pd.read_csv('data/demographic_sample.csv')

## 2. Cleaning Functions- Normalize column names- Clean string fields (State, District)- Parse dates- Fill missing numeric values

In [ ]:
def clean_dataframe(df, numeric_cols):    # 1. Normalize Header    df.columns = [c.lower().strip() for c in df.columns]        # 2. String Normalization    for col in ['state', 'district']:        if col in df.columns:            df[col] = df[col].astype(str).str.lower().str.strip().str.title()                # 3. Date Parsing    if 'date' in df.columns:        df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')            # 4. Fill NAs in numeric columns    # Ensure cols exist first (handling case sensitivity if header norm failed, but it shouldn't)    # Only fill specified numeric cols    numeric_cols = [c.lower() for c in numeric_cols]    for c in numeric_cols:        if c not in df.columns: df[c] = 0    df[numeric_cols] = df[numeric_cols].fillna(0)        return df

In [ ]:
cols_enc = ['age_0_5', 'age_5_17', 'age_18_greater']cols_bio = ['bio_age_5_17', 'bio_age_17_']cols_demo = ['demo_age_5_17', 'demo_age_17_']clean_enc = clean_dataframe(raw_enc.copy(), cols_enc)clean_bio = clean_dataframe(raw_bio.copy(), cols_bio)clean_demo = clean_dataframe(raw_demo.copy(), cols_demo)print("Data cleaning complete.")display(clean_enc.head(2))

## 3. AggregationGroup by State and Date to ensure uniqueness before merging.

In [ ]:
agg_enc = clean_enc.groupby(['state', 'date'])[ [c.lower() for c in cols_enc] ].sum().reset_index()agg_bio = clean_bio.groupby(['state', 'date'])[ [c.lower() for c in cols_bio] ].sum().reset_index()agg_demo = clean_demo.groupby(['state', 'date'])[ [c.lower() for c in cols_demo] ].sum().reset_index()print(f"Aggregated Shapes: Enc {agg_enc.shape}, Bio {agg_bio.shape}, Demo {agg_demo.shape}")

## 4. MergingOuter join all datasets to create a master timeline.

In [ ]:
master_df = pd.merge(agg_enc, agg_bio, on=['state', 'date'], how='outer')master_df = pd.merge(master_df, agg_demo, on=['state', 'date'], how='outer')master_df = master_df.fillna(0)print(f"Master Data Shape: {master_df.shape}")display(master_df.head())

## 5. Save Processed Data

In [ ]:
output_file = 'processed_master_data.csv'master_df.to_csv(output_file, index=False)print(f"Saved processed data to {output_file}")